In [1]:
#NMR superposition 
#Version 1.0.1

######USE NOTES######
# File type must be in CSV format with no titles.
# To obtain this from spinworks:
#    -Export spectrum as txt file, and replace (double space) with comma
#    -Delete the first line of file, so it starts with the first data point
#    -Rename file as "name".csv and choose "all file types"
# Select Ligand Data set first for labeling purposes. 
# Email warej21@up.edu with questions

######Release Notes######
# -1.0.0 - autoscale functionality working. Adjust the buffer to the left or right in "HARD SETTINGS" with var "BUFF"
# -1.0.1 - Auto Labeled Maximums
#        - Added Settings:
#            - Vertical Y axis modifications
#            - Line width
#            - Match Scale of Y axes

######TO-DO List######
# - Arrow showing shift
# - Fix flip-flopped LL/RL dictionaries

In [2]:
import numpy as np
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import os
import matplotlib.pyplot as plt

import tkinter

import matplotlib.animation as animation
from matplotlib.backend_bases import key_press_handler
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.figure import Figure

In [3]:
###HARD SETTINGS###

#Changes the fraction of the chart to the left & right of peaks
BUFFER = 0.3

#Offsets either the Lig or Coord x axis by vertModFactor
vertModLigandY = False
vertModCoordinatedY = False

vertModFactor = 0.1

#Changes the weight of the plotted lines
lineWidth = 0.5

#Forces the maximum values of both data sets to equal heights
matchYMax = False

In [4]:
Tk().withdraw()
filename1=askopenfilename(filetypes=[("CVS files", "*.csv")])
filename2=askopenfilename(filetypes=[("CVS files", "*.csv")])

In [5]:
DF1=pd.read_csv(filename1)
DF2=pd.read_csv(filename2)

In [6]:
#######

In [7]:
def _getMax(DFin):
    if DFin is DF1:
        DFx, DFy = np.loadtxt(filename1, usecols=[0, 1],
                      unpack=True, delimiter=',')
    else:
        DFx, DFy = np.loadtxt(filename2, usecols=[0, 1],
                      unpack=True, delimiter=',')
    DFmax=DFy.max()
    DFxList=DFx.tolist()
    DFyList=DFy.tolist()
    Index=DFyList.index(DFmax)
    XMax=DFxList[Index]
    return XMax, DFmax

In [8]:
def _compareLow(val1,val2):
    if val1>val2:
        return val2
    elif val1<val2:
        return val1
    else:
        return "ERROR"

In [9]:
def _compareHigh(val1,val2):
    if val1>val2:
        return val1
    elif val1<val2:
        return val2
    else:
        return "ERROR"

In [10]:
def _getScaleFactor():
    global XMaxDF1, XMaxDF2
    diff = abs(XMaxDF1-XMaxDF2)
    scaleFactor = diff*BUFFER
    return scaleFactor

In [11]:
def _getLL():
    global XMaxDF1, XMaxDF2, scaleFactor
    Lmax=_compareLow(XMaxDF1,XMaxDF2)
    LL=Lmax-scaleFactor
    return LL

In [12]:
def _getRL():
    global XMaxDF1, XMaxDF2, scaleFactor
    Rmax=_compareHigh(XMaxDF1,XMaxDF2)
    RL=Rmax+scaleFactor
    return RL

In [13]:
XMaxDF1, YMaxDF1=_getMax(DF1)

In [14]:
XMaxDF2, YMaxDF2=_getMax(DF2)

In [15]:
scaleFactor=_getScaleFactor()

In [16]:
RL=_getLL()

In [17]:
LL=_getRL()

In [18]:
####

In [19]:
root = tkinter.Tk()
root.wm_title("NMR Overlay")
current_list = ['Ligand', 'Coordinated']

In [20]:
fig = Figure(figsize=(2, 3), dpi=100)
ax = fig.add_subplot(111)

In [21]:
#Settings variables
vertMod=_compareHigh(YMaxDF1,XMaxDF1)*vertModFactor
scaleMatch= abs(_compareHigh(YMaxDF1,YMaxDF2))/_compareLow(YMaxDF1,YMaxDF2)
print(scaleMatch)

7.4859792970898456


In [22]:
for item in current_list:
    if item is "Ligand":
        x1, y1 = np.loadtxt(filename1, usecols=[0, 1],
                      unpack=True, delimiter=',')
        if matchYMax is True and _compareLow(YMaxDF1,YMaxDF2) is YMaxDF1:
            y1 *= scaleMatch
        #invx1= np.flip(x1)
        if vertModLigandY is True:
            ax.plot(x1, y1+vertMod, label="Ligand",linewidth=lineWidth)
        else:
            ax.plot(x1, y1, label="Ligand",linewidth=lineWidth)
    elif item is "Coordinated":
        x2, y2 = np.loadtxt(filename2, usecols=[0, 1],
                      unpack=True, delimiter=',')
        #invx2=np.flip(x2)
        if matchYMax is True and _compareLow(YMaxDF1,YMaxDF2)is YMaxDF2:
            y2 *= scaleMatch
        if vertModCoordinatedY is True:
            ax.plot(x2, y2+vertMod, label="Coordinated",linewidth=lineWidth)
        else:
            ax.plot(x2, y2, label="Coordinated",linewidth=lineWidth)

In [23]:
fig.legend()

ax.set_xlim(LL,RL)
ax.yaxis.set_visible(False)
ax.set_ylim(0,_compareHigh(YMaxDF1,YMaxDF2)*1.5)

canvas = FigureCanvasTkAgg(fig, master=root)
canvas.draw()
canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

toolbar = NavigationToolbar2Tk(canvas, root)
toolbar.update()
canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

In [24]:
#annotations
ax.annotate(XMaxDF1, xy=(XMaxDF1, YMaxDF1), xytext=(XMaxDF1, max(y1)+vertMod),ha="center")

ax.annotate(XMaxDF2, xy=(XMaxDF2, YMaxDF2), xytext=(XMaxDF2, max(y2)+vertMod),ha="center")

Text(2.86593, 7913899222.379101, '2.86593')

In [25]:
def on_key_press(event):
    print("you pressed {}".format(event.key))
    key_press_handler(event, canvas, toolbar)

    canvas.mpl_connect("key_press_event", on_key_press)

In [26]:
def _quit():
    root.quit()
    root.destroy()

In [27]:
quitButton = tkinter.Button(master=root, text="Quit", command=_quit)
quitButton.pack(side=tkinter.BOTTOM)

tkinter.mainloop()